In [15]:
import os
import sys
import ctypes
import math
import numpy
import numpy as np
import h5py
import tiledb

In [16]:
# Total number of time steps
# num_time_steps = 257
num_time_steps = 4

# Shape of the array for each time step
array_shape = (1024, 1024, 1024)

# Iterate over the .npy files and write to the HDF5 dataset
for timestep in range(1,num_time_steps):
    # Load the .npy file
    file_path = f'/home2/aoyagir/isotropic1024coarse/pressure2_time_steps/t_{timestep:04}.npy'
    if os.path.exists(file_path):
        # print("iine")
        pass
    else:
        print("doesnt exists")

In [12]:
# Create a TileDB config
config = tiledb.Config()

# Set Minio-specific configuration options
 

config["vfs.s3.scheme"] = "http"
config["vfs.s3.region"] = ""
config["vfs.s3.endpoint_override"] = "172.20.2.253:9000"
config["vfs.s3.use_virtual_addressing"] = "false"
config["vfs.s3.aws_access_key_id"] = "189XB7837GTE0Kt1lv6b"
config["vfs.s3.aws_secret_access_key"] = "aQ3nGhtwk8Qod24f201KAMzShU5Y5VUgVOQL0XG5"

# Create contex
ctx = tiledb.Ctx(config)



In [20]:
array_name = f"array{num_time_steps}"
# array_name = f"array_{num_time_steps}lz4"
# array_name = f"array_{num_time_steps}zstd"
# array_name = f"array_{num_time_steps}gzip"


# array_uri = f"/home2/aoyagir/tiledb_data/{array_name}"
array_uri = f"s3://{array_name}"
print(array_name)

array_schema = tiledb.ArraySchema(
    domain=tiledb.Domain(
        tiledb.Dim(name="timestep",domain=(0, num_time_steps - 1), tile=1, dtype=np.uint64),
        tiledb.Dim(name="x", domain=(0, 1023), tile=256, dtype=np.uint64),
        tiledb.Dim(name="y", domain=(0, 1023), tile=256, dtype=np.uint64),
        tiledb.Dim(name="z", domain=(0, 1023), tile=256, dtype=np.uint64),
    ),
    attrs=[tiledb.Attr(name="data", dtype=np.float32)],
    # filters=tiledb.FilterList([tiledb.LZ4Filter()]),  # Enable LZ4 compression
    # filters=tiledb.FilterList([gzip_filter])
    # filters=tiledb.FilterList([zstd_filter])
    # filters=tiledb.FilterList([blosc_filter])
)

tiledb.Array.create(array_uri, array_schema,ctx=ctx)

array4


In [21]:
# Iterate over the .npy files and write to the TileDB dataset
for timestep in range(1, num_time_steps):
    file_path = f'/home2/aoyagir/isotropic1024coarse/pressure2_time_steps/t_{timestep:04}.npy'
    if os.path.exists(file_path):
        # Load the .npy file
        data_3d = np.load(file_path).astype(np.float32)

        # Open the TileDB array for writing
        with tiledb.DenseArray(array_uri, mode="w", ctx=ctx) as array:
            # Define the coordinates for this timestep
            coords = (timestep-1, slice(0, 1024), slice(0, 1024), slice(0, 1024))
            # Write the 3D data to the TileDB array
            array[coords] = data_3d
        print(f"timestep{timestep} is done")
    else:
        print(f"File doesn't exist for timestep {timestep}")